In [ ]:
import pandas as pd
# read the CSV file
dataset= pd.read_csv('/content/deliveries.csv')

# to see the dataset columns
print('dataset columns:')
print(dataset.columns.unique())

# checking the first 5 rows in a dataset
dataset.head()

dataset columns:
Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')


,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


In [ ]:
# calculate the key metrics for each batter using groupby function
# total runs scored by each player
total_runs=dataset.groupby('batter')['batsman_runs'].sum()
total_runs

,batsman_runs
batter,
A Ashish Reddy,280
A Badoni,634
A Chandila,4
A Chopra,53
A Choudhary,25
...,...
Yashpal Singh,47
Younis Khan,3
Yudhvir Singh,22


In [ ]:
# total balls faced by each batter
balls_faced=dataset.groupby('batter')['batsman_runs'].count()
balls_faced

,batsman_runs
batter,
A Ashish Reddy,196
A Badoni,505
A Chandila,7
A Chopra,75
A Choudhary,20
...,...
Yashpal Singh,67
Younis Khan,7
Yudhvir Singh,16


In [ ]:
# total dismissal for each batter
dismissals=dataset['player_dismissed'].value_counts()
dismissals

,count
player_dismissed,
RG Sharma,223
V Kohli,207
S Dhawan,193
KD Karthik,184
RV Uthappa,180
...,...
BJ Haddin,1
SS Cottrell,1
JE Taylor,1


In [ ]:
# count of hundred and less than hundreds for each batter
innings_scores_each_batter=dataset.groupby('batter')['batsman_runs'].sum()
print(f'innings scores for each batter:\n {innings_scores_each_batter}')
hundreds= innings_scores_each_batter[innings_scores_each_batter >= 100].groupby('batter').count()
print(f'hundred scores by each batter:\n {hundreds}')
less_than_hundreds= innings_scores_each_batter[(innings_scores_each_batter < 100) & (innings_scores_each_batter >=50)].groupby('batter').count()
print(f'less than hundreds scores by each batter:\n {less_than_hundreds}')

innings scores for each batter:
 batter
A Ashish Reddy     280
A Badoni           634
A Chandila           4
A Chopra            53
A Choudhary         25
                  ... 
Yashpal Singh       47
Younis Khan          3
Yudhvir Singh       22
Yuvraj Singh      2754
Z Khan             117
Name: batsman_runs, Length: 673, dtype: int64
hundred scores by each batter:
 batter
A Ashish Reddy    1
A Badoni          1
A Manohar         1
A Mishra          1
A Raghuvanshi     1
                 ..
YBK Jaiswal       1
YK Pathan         1
YV Takawale       1
Yuvraj Singh      1
Z Khan            1
Name: batsman_runs, Length: 304, dtype: int64
less than hundreds scores by each batter:
 batter
A Chopra          1
A Flintoff        1
AA Bilakhia       1
AD Mascarenhas    1
AD Nath           1
                 ..
Vishnu Vinod      1
Vivrant Sharma    1
WA Mota           1
WD Parnell        1
WPUJC Vaas        1
Name: batsman_runs, Length: 73, dtype: int64


In [ ]:
# combine these into a single dataframe and fill missing values with 0
batting_stats=pd.DataFrame({
    'total_runs':total_runs,
    'balls_faced':balls_faced,
    'dismissals':dismissals,
    '100s':hundreds,
    '50s':less_than_hundreds}
).fillna(0)
batting_stats

,total_runs,balls_faced,dismissals,100s,50s
A Ashish Reddy,280.0,196.0,15.0,1.0,0.0
A Badoni,634.0,505.0,26.0,1.0,0.0
A Chandila,4.0,7.0,0.0,0.0,0.0
A Chopra,53.0,75.0,6.0,0.0,1.0
A Choudhary,25.0,20.0,1.0,0.0,0.0
...,...,...,...,...,...
Yashpal Singh,47.0,67.0,4.0,0.0,0.0
Younis Khan,3.0,7.0,1.0,0.0,0.0
Yudhvir Singh,22.0,16.0,4.0,0.0,0.0
Yuvraj Singh,2754.0,2207.0,111.0,1.0,0.0


In [ ]:
# calculate average abd strike rate
batting_stats['average']=batting_stats['total_runs']/batting_stats['dismissals'].replace(0,1)
batting_stats['strike_rate']=batting_stats['total_runs']/batting_stats['balls_faced']*100

In [ ]:
# normalize the metrics(0-100 scale)
batting_stats['normalized_average']=(batting_stats['average']/batting_stats['average'].max())*100
batting_stats['normalized_strike_rate']=(batting_stats['strike_rate']/batting_stats['strike_rate'].max())*100
batting_stats['normalized 100s']=(batting_stats['100s']/batting_stats['100s'].max())*100
batting_stats['normalized 50s']=(batting_stats['50s']/batting_stats['50s'].max())*100

In [ ]:
# Calculate Overall Score using weights

weights = {'average': 0.3, 'strike_rate': 0.4, '100s': 0.15, '50s': 0.05}
batting_stats['Overall Score'] = (
(batting_stats['normalized_average']* weights['average']) +
(batting_stats['normalized_strike_rate'] * weights['strike_rate']) +
(batting_stats['normalized 100s'] * weights['100s']) +
(batting_stats['normalized 50s'] * weights['50s'])
)

In [ ]:
# rank players by overall scores
batting_stats_sorted=batting_stats.sort_values(by='Overall Score',ascending=False)
batting_stats_sorted

,total_runs,balls_faced,dismissals,100s,50s,average,strike_rate,normalized_average,normalized_strike_rate,normalized 100s,normalized 50s,Overall Score
J Fraser-McGurk,330.0,150.0,9.0,1.0,0.0,36.666667,220.000000,53.140097,73.333333,100.0,0.0,60.275362
MN van Wyk,167.0,135.0,3.0,1.0,0.0,55.666667,123.703704,80.676329,41.234568,100.0,0.0,55.696726
DP Conway,924.0,669.0,19.0,1.0,0.0,48.631579,138.116592,70.480549,46.038864,100.0,0.0,54.559710
Vivrant Sharma,69.0,49.0,1.0,0.0,1.0,69.000000,140.816327,100.000000,46.938776,0.0,100.0,53.775510
T Stubbs,405.0,239.0,11.0,1.0,0.0,36.818182,169.456067,53.359684,56.485356,100.0,0.0,53.602047
...,...,...,...,...,...,...,...,...,...,...,...,...
RP Meredith,0.0,3.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
RR Bhatkal,0.0,2.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
S Ladda,0.0,10.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000
U Kaul,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


In [ ]:
# calculate mean,median and mode for runs
runs_mean=batting_stats_sorted['total_runs'].mean()
runs_median=batting_stats_sorted['total_runs'].median()
runs_mode=batting_stats_sorted['total_runs'].mode()
print(f'mean: {runs_mean}')
print(f'median: {runs_median}')
print(f'mode: {runs_mode}')

mean: 489.7091988130564
median: 74.5
mode: 0    0.0
Name: total_runs, dtype: float64


In [ ]:
# calculate mean,median and mode for wickets
matchwise_wickets=dataset.groupby('match_id')['player_dismissed'].apply(lambda x: x.notna().sum())
wickets_mean=matchwise_wickets.mean()
wickets_median=matchwise_wickets.median()
wickets_mode=matchwise_wickets.mode().iloc[0] if not matchwise_wickets.mode().empty else 0
print(f'mean: {wickets_mean}')
print(f'median: {wickets_median}')
print(f'mode: {wickets_mode}')

mean: 11.826484018264841
median: 12.0
mode: 13


In [ ]:
# calculate runs and wickets for each player
player_runs=dataset.groupby('batter')['batsman_runs'].sum() # total runs scored by each player
wickets_per_player=dataset.groupby('bowler')['player_dismissed'].apply(lambda x: x.notna().sum())
print(f' runs of player:\n {player_runs}')
print(f'wickets per player:\n {wickets_per_player}')

 runs of player:
 batter
A Ashish Reddy     280
A Badoni           634
A Chandila           4
A Chopra            53
A Choudhary         25
                  ... 
Yashpal Singh       47
Younis Khan          3
Yudhvir Singh       22
Yuvraj Singh      2754
Z Khan             117
Name: batsman_runs, Length: 673, dtype: int64
wickets per player:
 bowler
A Ashish Reddy     19
A Badoni            2
A Chandila         11
A Choudhary         5
A Dananjaya         0
                 ... 
Yash Dayal         31
Yash Thakur        27
Yudhvir Singh       4
Yuvraj Singh       39
Z Khan            119
Name: player_dismissed, Length: 530, dtype: int64


In [ ]:
# calculate mean, median and mode for runs for each player
runs_mean= player_runs.mean()
runs_median=player_runs.median()
runs_mode=player_runs.mode().iloc[0] if not player_runs.mode().empty else 0
print(f'mean: {runs_mean}')
print(f'median: {runs_median}')
print(f'mode: {runs_mode}')

mean: 490.4368499257058
median: 75.0
mode: 0


In [ ]:
# calculate mean, median and mode for wickets for each player
wickets_mean=wickets_per_player.mean()
wickets_median= wickets_per_player.median()
wickets_mode= wickets_per_player.mode().iloc[0] if not wickets_per_player.mode().empty else 0
print(f'mean: {wickets_mean}')
print(f'median: {wickets_median}')
print(f'mode: {wickets_mode}')

mean: 24.433962264150942
median: 8.5
mode: 0


In [ ]:
# creating a dataframe stats for each player
runs_stats= player_runs.describe().to_frame().T
wickets_stats=wickets_per_player.describe().to_frame().T
print(f'statistics of runs :\n{runs_stats}')
print(f'statistics of wickets :\n{wickets_stats}')

statistics of runs :
              count       mean          std  min   25%   50%    75%     max
batsman_runs  673.0  490.43685  1041.075277  0.0  15.0  75.0  360.0  8014.0
statistics of wickets :
                  count       mean        std  min  25%  50%    75%    max
player_dismissed  530.0  24.433962  38.896406  0.0  2.0  8.5  27.75  213.0


In [ ]:
combined_stats_of_runs_and_wickets= pd.concat([runs_stats[['mean','50%','std']],wickets_stats[['mean','50%','std']]], axis=1)
print(f'combined statistics of runs and wickets:\n{combined_stats_of_runs_and_wickets}')

combined statistics of runs and wickets:
                       mean   50%          std       mean  50%        std
batsman_runs      490.43685  75.0  1041.075277        NaN  NaN        NaN
player_dismissed        NaN   NaN          NaN  24.433962  8.5  38.896406


In [ ]:
import pandas as pd

Matches= pd.read_csv('/content/matches.csv')

Matches

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,1426307,2024,Hyderabad,2024-05-19,League,Abhishek Sharma,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Punjab Kings,Sunrisers Hyderabad,Punjab Kings,bat,Sunrisers Hyderabad,wickets,4.0,215.0,20.0,N,NaN,Nitin Menon,VK Sharma
1091,1426309,2024,Ahmedabad,2024-05-21,Qualifier 1,MA Starc,"Narendra Modi Stadium, Ahmedabad",Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad,bat,Kolkata Knight Riders,wickets,8.0,160.0,20.0,N,NaN,AK Chaudhary,R Pandit
1092,1426310,2024,Ahmedabad,2024-05-22,Eliminator,R Ashwin,"Narendra Modi Stadium, Ahmedabad",Royal Challengers Bengaluru,Rajasthan Royals,Rajasthan Royals,field,Rajasthan Royals,wickets,4.0,173.0,20.0,N,NaN,KN Ananthapadmanabhan,MV Saidharshan Kumar
1093,1426311,2024,Chennai,2024-05-24,Qualifier 2,Shahbaz Ahmed,"MA Chidambaram Stadium, Chepauk, Chennai",Sunrisers Hyderabad,Rajasthan Royals,Rajasthan Royals,field,Sunrisers Hyderabad,runs,36.0,176.0,20.0,N,NaN,Nitin Menon,VK Sharma


In [ ]:
# Find A list of the Top 20 Cities where the most number of matches have been played.

# Assuming 'Matches' DataFrame is already loaded as shown in the provided code.

top_cities = Matches['venue'].value_counts().head(20)
top_cities

,count
venue,
Eden Gardens,77
Wankhede Stadium,73
M Chinnaswamy Stadium,65
Feroz Shah Kotla,60
"Rajiv Gandhi International Stadium, Uppal",49
"MA Chidambaram Stadium, Chepauk",48
Sawai Mansingh Stadium,47
Dubai International Cricket Stadium,46
"Wankhede Stadium, Mumbai",45


In [ ]:
matches_played =Matches.groupby('venue')['id'].nunique()
top_venue = matches_played.sort_values(ascending=False)
top_venue
# It is showing that eden gardens host maximum no of matches of IPL.

,id
venue,
Eden Gardens,77
Wankhede Stadium,73
M Chinnaswamy Stadium,65
Feroz Shah Kotla,60
"Rajiv Gandhi International Stadium, Uppal",49
"MA Chidambaram Stadium, Chepauk",48
Sawai Mansingh Stadium,47
Dubai International Cricket Stadium,46
"Wankhede Stadium, Mumbai",45


In [ ]:
import pandas as pd

# Assuming 'Matches' DataFrame is already loaded

# Get unique venues and sort them alphabetically
unique_venues = sorted(Matches['venue'].unique())

# Get the total count
venue_count = len(unique_venues)

# Print the results
print("Total number of venues:", venue_count)
print("\nVenues in alphabetical order:")
for venue in unique_venues:
    print(venue)

Total number of venues: 58

Venues in alphabetical order:
Arun Jaitley Stadium
Arun Jaitley Stadium, Delhi
Barabati Stadium
Barsapara Cricket Stadium, Guwahati
Bharat Ratna Shri Atal Bihari Vajpayee Ekana Cricket Stadium, Lucknow
Brabourne Stadium
Brabourne Stadium, Mumbai
Buffalo Park
De Beers Diamond Oval
Dr DY Patil Sports Academy
Dr DY Patil Sports Academy, Mumbai
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium
Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium, Visakhapatnam
Dubai International Cricket Stadium
Eden Gardens
Eden Gardens, Kolkata
Feroz Shah Kotla
Green Park
Himachal Pradesh Cricket Association Stadium
Himachal Pradesh Cricket Association Stadium, Dharamsala
Holkar Cricket Stadium
JSCA International Stadium Complex
Kingsmead
M Chinnaswamy Stadium
M Chinnaswamy Stadium, Bengaluru
M.Chinnaswamy Stadium
MA Chidambaram Stadium
MA Chidambaram Stadium, Chepauk
MA Chidambaram Stadium, Chepauk, Chennai
Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur
M

In [ ]:
#  How to combine the same names in the venue

import pandas as pd

# Assuming 'Matches' DataFrame is already loaded as shown in the provided code.
Matches = pd.read_csv('/content/matches.csv')

# Group by city and count the number of matches
venue_counts = Matches['venue'].value_counts()

# Create a dictionary to map similar venue names to a single name
venue_mapping = {
    'M Chinnaswamy Stadium': 'M.Chinnaswamy Stadium',
    'M.Chinnaswamy Stadium': 'M.Chinnaswamy Stadium',
    'Wankhede Stadium': 'Wankhede Stadium',
    # Add more mappings as needed based on your data
    # Example: if 'Punjab Cricket Association IS Bindra Stadium' sometimes appears as 'IS Bindra Stadium, Mohali'
    # 'Punjab Cricket Association IS Bindra Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
    # ... and so on
}

# Replace venue names using the mapping
Matches['venue'] = Matches['venue'].replace(venue_mapping)

# Recalculate the venue counts after combining similar names
updated_venue_counts = Matches['venue'].value_counts()


# Display the top 20 venues after combining similar names
print(updated_venue_counts.head(20))

venue
M.Chinnaswamy Stadium                         80
Eden Gardens                                  77
Wankhede Stadium                              73
Feroz Shah Kotla                              60
Rajiv Gandhi International Stadium, Uppal     49
MA Chidambaram Stadium, Chepauk               48
Sawai Mansingh Stadium                        47
Dubai International Cricket Stadium           46
Wankhede Stadium, Mumbai                      45
Punjab Cricket Association Stadium, Mohali    35
Sheikh Zayed Stadium                          29
Sharjah Cricket Stadium                       28
MA Chidambaram Stadium, Chepauk, Chennai      28
Narendra Modi Stadium, Ahmedabad              24
Maharashtra Cricket Association Stadium       22
Dr DY Patil Sports Academy, Mumbai            20
Dr DY Patil Sports Academy                    17
Brabourne Stadium, Mumbai                     17
Eden Gardens, Kolkata                         16
Subrata Roy Sahara Stadium                    16
Name: count, d

In [ ]:
len(Matches['venue'].unique())

58

In [ ]:
#How to combine the same names in the venue

# Assuming 'Matches' DataFrame is already loaded as shown in the provided code.
Matches = pd.read_csv('/content/matches.csv')

# Create a dictionary to map similar venue names to a single name
# This dictionary needs to be comprehensive based on your data
venue_mapping = {
    'M Chinnaswamy Stadium': 'M.Chinnaswamy Stadium',
    'M.Chinnaswamy Stadium': 'M.Chinnaswamy Stadium',
    'Wankhede Stadium': 'Wankhede Stadium',
    'Rajiv Gandhi International Stadium, Uppal': 'Rajiv Gandhi International Stadium',
    'MA Chidambaram Stadium, Chepauk': 'MA Chidambaram Stadium',
    'Feroz Shah Kotla': 'Arun Jaitley Stadium',
    'Arun Jaitley Stadium': 'Arun Jaitley Stadium',
    'Punjab Cricket Association IS Bindra Stadium, Mohali':'Punjab Cricket Association IS Bindra Stadium',
    'Punjab Cricket Association Stadium, Mohali': 'Punjab Cricket Association IS Bindra Stadium',
    'Sawai Mansingh Stadium': 'Sawai Mansingh Stadium',
    'Eden Gardens': 'Eden Gardens',
    'Holkar Cricket Stadium': 'Holkar Cricket Stadium',
    'Subrata Roy Sahara Stadium': 'Maharashtra Cricket Association Stadium',
    'Maharashtra Cricket Association Stadium': 'Maharashtra Cricket Association Stadium',
    'Dr DY Patil Sports Academy': 'Dr DY Patil Sports Academy',
    'Kingsmead': 'Kingsmead',
    'Newlands': 'Newlands',
    'St George\'s Park': 'St George\'s Park',
    'SuperSport Park': 'SuperSport Park',
    'Buffalo Park': 'Buffalo Park',
    'New Wanderers Stadium': 'New Wanderers Stadium',
    'De Beers Diamond Oval': 'De Beers Diamond Oval',
    # Add more mappings as needed based on your data
}

# Replace venue names using the mapping
Matches['venue'] = Matches['venue'].replace(venue_mapping)

# Recalculate the venue counts after combining similar names
updated_venue_counts = Matches['venue'].value_counts()

# Display the top 20 venues after combining similar names
print(updated_venue_counts.head(20))
print(len(Matches['venue'].unique()))

venue
M.Chinnaswamy Stadium                           80
Eden Gardens                                    77
Arun Jaitley Stadium                            74
Wankhede Stadium                                73
Rajiv Gandhi International Stadium              64
MA Chidambaram Stadium                          57
Punjab Cricket Association IS Bindra Stadium    56
Sawai Mansingh Stadium                          47
Dubai International Cricket Stadium             46
Wankhede Stadium, Mumbai                        45
Maharashtra Cricket Association Stadium         38
Sheikh Zayed Stadium                            29
MA Chidambaram Stadium, Chepauk, Chennai        28
Sharjah Cricket Stadium                         28
Narendra Modi Stadium, Ahmedabad                24
Dr DY Patil Sports Academy, Mumbai              20
Brabourne Stadium, Mumbai                       17
Dr DY Patil Sports Academy                      17
Eden Gardens, Kolkata                           16
Arun Jaitley Stadium, Del